In [1]:
import talib
# import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5
import time

In [2]:
if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
login = 114781990
password = 'Mypassword$1234'
server = 'Exness-MT5Trial6'

mt5.login(login, password, server)

True

In [27]:
# For getting price at 40th second of 15 MIN

import datetime

# Get the current time
while True:
    current_time = datetime.datetime.now()

    # Check if the second value is equal to 40
    if current_time.second == 40:
        
        rate = mt5.copy_rates_from('EURUSDm', mt5.TIMEFRAME_M15, datetime.datetime.now(), 1)

        df = pd.DataFrame(rate)


        df['date']=pd.to_datetime(df['time'], unit='s')

        print(df.head())

        break


         time     open     high      low    close  tick_volume  spread  \
0  1693839600  1.07839  1.07879  1.07832  1.07867          153       8   

   real_volume                date  
0            0 2023-09-04 15:00:00  


In [35]:
# Calculate Keltner's Channel

def get_kc( high, low, close, kc_lookback, multiplier, atr_lookback):
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift()))
    tr3 = pd.DataFrame(abs(low - close.shift()))

    # Calculate True Range as the element-wise maximum of the three components
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.ewm(alpha=1/atr_lookback).mean()

    kc_middle = close.ewm(span=kc_lookback).mean()
    kc_upper = kc_middle + multiplier * atr
    kc_lower = kc_middle - multiplier * atr

    return kc_middle, kc_upper, kc_lower

In [32]:
# # Getting data


# rate = mt5.copy_rates_from('EURUSDm', mt5.TIMEFRAME_M15, datetime.datetime.now(), 1000)
# df = pd.DataFrame(rate)


# df['date']=pd.to_datetime(df['time'], unit='s')

# df.to_excel('training.xlsx')

In [44]:
def check_prev_candle( element):

    current_index = df.loc[df.date ==  element.date].index[0]
    current_close = element.close
    current_candle = element.candle



    if current_index > 0:
        prev_index = current_index - 1 
    #           prev candle colour
        prev_candle = df.iloc[prev_index]['candle']

    #           current candle colour
        current_candle = df.iloc[current_index]['candle']

        if current_candle == 'G':
    #                 Both green current close must be higher
            if prev_candle == 'G' and df.iloc[current_index]['close'] > df.iloc[prev_index][ 'close']:
                return 1

    #                If last candle Red then entry candle must close above last candle open
            elif prev_candle == 'R' and df.iloc[current_index]['close'] > df.iloc[prev_index][ 'open']:
                return 1

            else:
                return 0

        elif current_candle == 'R':
    #                 Both red current close must be lower
            if prev_candle == 'R' and df.iloc[current_index]['close'] < df.iloc[prev_index][ 'close']:
                return 1

    #                If last candle green then entry candle must close below last candle open
            elif prev_candle == 'G' and df.iloc[current_index]['close'] < df.iloc[prev_index][ 'open']:
                return 1

            else:
                return 0

    #     How to check for the 1st candle?
    return 0


def candle( df):
    '''Return candle colour'''

    if df['close'] - df['open'] > 0:
        return 'G'

    else:
        return 'R'


In [45]:
# Making all indicators in training 
# Testing all logic on testing.xlsx

df = pd.read_excel('./training.xlsx', engine='openpyxl')

# Record the start time
start_time = datetime.datetime.now()

df['EMA_20'],df['upper_keltner'],df['lower_keltner'] = get_kc(df['high'], df['low'], df['close'], kc_lookback = 20, multiplier = 2.25, atr_lookback = 10)
df['candle'] = df.apply(candle, axis=1)
df['prev'] = df.apply(check_prev_candle, axis=1)
# Record the end time
end_time = datetime.datetime.now()

# Calculate the elapsed time
elapsed_time = end_time - start_time


print(f"Your function took {elapsed_time.total_seconds():.4f} seconds to run.")

Your function took 0.5474 seconds to run.


In [46]:
df.head()

,Unnamed: 0,date,open,high,low,close,EMA_20,upper_keltner,lower_keltner,candle,prev
0,0,2023-08-18 16:30:00,1.08731,1.08751,1.08709,1.08717,1.087170,1.088115,1.086225,R,0
1,1,2023-08-18 16:45:00,1.08717,1.08736,1.08677,1.08727,1.087222,1.088369,1.086076,G,0
2,2,2023-08-18 17:00:00,1.08726,1.08803,1.08726,1.08773,1.087409,1.088771,1.086046,G,1
3,3,2023-08-18 17:15:00,1.08773,1.08773,1.08680,1.08717,1.087340,1.088915,1.085765,R,1
4,4,2023-08-18 17:30:00,1.08717,1.08759,1.08707,1.08758,1.087398,1.088874,1.085922,G,0


In [15]:
class BackTest:
    
    def __init__(self, instrument):
        
#         self.df = pd.DataFrame(yf.download(tickers=instrument, start = '2022-10-01', end = '2023-06-01', interval='1h'))
        if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
        login = 114781990
        password = 'Mypassword$1234'
        server = 'Exness-MT5Trial6'

        mt5.login(login, password, server)
        print('Working on: ', instrument[0])
#         self.df = pd.read_excel('./training.xlsx', engine='openpyxl')
        rate = mt5.copy_rates_from(instrument[0], mt5.TIMEFRAME_M15, datetime.datetime.now(), instrument[2])
        self.df = pd.DataFrame(rate)
        
         
        self.df['date']=pd.to_datetime(self.df['time'], unit='s')

        
#         self.analyse = pd.DataFrame(columns=['TOTAL TRADES', 'WINS', 'LOSSES', 'PERFORMANCE', 'WIN RATE', 
#                                 'WINNING STREAK', 'LOSING STREAK', 'MAX DRAWDOWN', 'AVG GAIN', 'AVG LOSS'])
        
        '''Renaming columns and deleting unnecessary columns'''

        self.df.set_index(np.arange(len(self.df)), inplace = True)
        self.df = self.df.drop(['spread', 'real_volume', 'tick_volume'], axis='columns')
        
        self.df.to_excel('EURUSD.xlsx')
        
        
        self.one_pip_value = instrument[1]
        self.account = 10000
#         print(self.df.head())
        
    def display(self):
        
#         print(self.df.head())
#         self.result.to_excel('Forex.xlsx')
        self.squeeze.to_excel('EURUSD.xlsx')
#         print(self.analyse)
#         return(self.analyse)
    
    
    def in_squeeze(self, df):
        return df['lower_band'] > df['lower_keltner'] and df['upper_band'] < df['upper_keltner']
    
    
    def define_trend(self, current_high, current_low, prev_high, prev_low, days, i):
        #       Uptrend
            if current_high > prev_high and current_low > prev_low:
    #           Find point at which current sample
    #           crossed prev high

                stream = self.df.loc[self.df.index[i : i+days], 'high']
                up_candle_index = stream.loc[stream > prev_high].index[0]
                
    #             if df.loc[i+15, 'high'] > prev_high:
                    
                self.df.loc[up_candle_index : i+days, 'trend'] = 'up'

            
                if self.df.iloc[i-1]['trend'] == 'up':
                    self.df.loc[i: up_candle_index, 'trend'] = 'up'

                else:
                    self.df.loc[i: up_candle_index, 'trend'] = 'no trend'


    #         Downtrend
            elif (current_high < prev_high) and (current_low < prev_low):
    #           Find point at which current sample 
    #           crossed prev low

                stream = self.df.loc[self.df.index[i : i+days], 'low']
                down_candle_index = stream.loc[stream < prev_low,].index[0]
               
#                 print('DownCandleIndex: ', down_candle_index)
#                 print('Days Callee: ', days)
                
                self.df.loc[down_candle_index : i+days, 'trend'] = 'down'

                if self.df.iloc[i-1]['trend'] == 'down':
                    self.df.loc[i: down_candle_index, 'trend'] = 'down'

                else:
                    self.df.loc[i: down_candle_index, 'trend'] = 'no trend'


                    
            else:
                self.df.loc[i : i+days, 'trend'] = 'no trend'

    
    def get_trend(self, i=0):
        
        length = len(self.df)

        # print('Length: ', length)
        
        days = 21

        prev_high = self.df.loc[self.df.index[i : i+days], 'high'].max()
        
        prev_low = self.df.loc[self.df.index[i : i+days], 'low'].min()

        while(i + days  < length):

            # print('In Trend !!')
        
            i += days
            
            current_high = self.df.loc[self.df.index[i : i+days], 'high'].max()
            current_low = self.df.loc[self.df.index[i : i+days], 'low'].min()
            
            self.define_trend(current_high, current_low, prev_high, prev_low, days, i)
            
            prev_high = current_high
            prev_low = current_low
            
                    
                
        if i < length-1:
            prev_high = self.df.loc[self.df.index[i-days:i], 'high'].max()
            prev_low = self.df.loc[self.df.index[i-days:i], 'low'].min()
            
            current_high = self.df.loc[self.df.index[i : length], 'high'].max()
            current_low = self.df.loc[self.df.index[i : length], 'low'].min()
            
            days = length-i
            
            self.define_trend(current_high, current_low, prev_high, prev_low, days, i)
            
            
        

    # KELTNER CHANNEL CALCULATION

    def get_kc(self, high, low, close, kc_lookback, multiplier, atr_lookback):
        tr1 = pd.DataFrame(high - low)
        tr2 = pd.DataFrame(abs(high - close.shift()))
        tr3 = pd.DataFrame(abs(low - close.shift()))
        
        # Calculate True Range as the element-wise maximum of the three components
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.ewm(alpha=1/atr_lookback).mean()

        kc_middle = close.ewm(span=kc_lookback).mean()
        kc_upper = kc_middle + multiplier * atr
        kc_lower = kc_middle - multiplier * atr

        return kc_middle, kc_upper, kc_lower
    
    
    def indicators(self):
        '''Adding EMAs, BB, Keltner Channel and RSI'''
        
        self.df['EMA_8'] = talib.EMA(self.df['close'], timeperiod=8)

        # EMA 20 group
#         self.df['EMA_15'] = talib.EMA(self.df['close'], timeperiod=15)
        self.df['EMA_20'] = talib.EMA(self.df['close'], timeperiod=20)
#         self.df['EMA_22'] = talib.EMA(self.df['close'], timeperiod=22)
        # EMA 50 group
#         self.df['EMA_48'] = talib.EMA(self.df['close'], timeperiod=48)
#         self.df['EMA_50'] = talib.EMA(self.df['close'], timeperiod=50)
#         self.df['EMA_52'] = talib.EMA(self.df['close'], timeperiod=52)
        # EMA 200 group
#         self.df['EMA_198'] = talib.EMA(self.df['close'], timeperiod=198)
        self.df['EMA_200'] = talib.EMA(self.df['close'], timeperiod=200)
#         self.df['EMA_202'] = talib.EMA(self.df['close'], timeperiod=202)

        self.df['RSI'] = talib.RSI(self.df['close'], timeperiod=21)
        self.df = self.df.dropna()

        self.df.set_index(np.arange(len(self.df)), inplace = True)
        # Identify trend on 21 days highs & lows
        self.get_trend()
#         self.df.to_excel('check_trend.xlsx')

    #     df.head()

        '''Bollinger Band'''
        self.df['20sma'] = self.df['close'].rolling(window=20).mean()
        self.df['stddev'] = self.df['close'].rolling(window=20).std()
        self.df['lower_band'] = self.df['20sma'] - (2 * self.df['stddev'])
        self.df['upper_band'] = self.df['20sma'] + (2 * self.df['stddev'])

        '''Keltners Channel'''
        self.df['TR'] = abs(self.df['high'] - self.df['low'])
        self.df['ATR'] = self.df['TR'].rolling(window=20).mean()

        self.df['lower_keltner'] = self.df['20sma'] - (self.df['ATR'] * 1.2)
        self.df['upper_keltner'] = self.df['20sma'] + (self.df['ATR'] * 1.2)
        
        
        self.df['squeeze_on'] = self.df.apply(self.in_squeeze, axis=1)

        self.df = self.df.drop(['stddev', 'lower_band', 'upper_band', '20sma', 'TR', 'ATR', 'lower_keltner', 'upper_keltner'],axis=1)

        '''Get Keltner for trading'''
        
        self.df['EMA_20'],self.df['upper_keltner'],self.df['lower_keltner'] = self.get_kc(self.df['high'], self.df['low'], self.df['close'], kc_lookback = 20, multiplier = 2.25, atr_lookback = 10)
        
        self.df = self.df.dropna()

        self.df.set_index(np.arange(len(self.df)), inplace = True)

        self.df.index.name='sn'
        # print('Updated DF: ',self.df.head())
        
#         self.df.to_excel('check_trend.xlsx')
        
    
    def candle(self, df):
        '''Return candle colour'''

        if df['close'] - df['open'] > 0:
            return 'G'

        else:
            return 'R'
    

    def check_body(self, df):
        #     Calculate 0.7 % of entry price
    #     fOR 1H FOREX
            one_percent = df['open']/100 
            percent = one_percent * 0.06
            hammer_per = one_percent * 0.06


        #     Calculate candle size close-open
            if df['candle'] == 'G':
                candle_size = df['close'] - df['open']
                upper_wick = df['high'] - df['close']
                lower_wick = df['open'] - df['low']

            else:
                candle_size = abs(df['close'] - df['open'])
                upper_wick = df['high'] - df['open']
                lower_wick = df['close'] - df['low']

            upper_wick_per = (upper_wick/one_percent) 
            lower_wick_per = (lower_wick/one_percent) 



        #     check for entry candle size and Max candle size 4.5 %
            if ((candle_size >= percent) and candle_size <= (4.5 * one_percent) and
             (candle_size > upper_wick) and (candle_size > lower_wick)) and (upper_wick_per < 0.06) and (lower_wick_per < 0.06):
                
                if df['candle'] == 'G' and df['high'] < df['upper_keltner'] :

                    return 1
                
                elif df['candle'] == 'R' and df['low'] > df['lower_keltner'] :

                    return 1
                
                else:
                    
                    return 0

        #     condition for hammmer
        #     lower wick is atleast 50 % of total size 
        #     Green Hammer 
        # need to add max wick of opposite dir
            elif ( (df['candle'] == 'G') and (df['open'] - df['low']) >= ((df['high'] - df['low']) * 0.5) and
                  (candle_size >= hammer_per) and (candle_size > upper_wick)and
                  candle_size <= (4.5 * one_percent) and ((upper_wick_per < 0.06))  
                 and df['high'] < df['upper_keltner']):

                return 1


    #     For Red Hammer
    # need to add max wick of opposite dir
            elif ( (df['candle'] == 'R') and (df['high'] - df['open']) >= (abs(df['high'] - df['low']) * 0.5) and 
                  (candle_size >= hammer_per) and (candle_size > lower_wick)and candle_size <= (4.5 * one_percent) and 
                  ((lower_wick_per < 0.06)) and df['low'] > df['lower_keltner'] ):

                return 1


            else:
                return 0

        
    
    def check_prev_candle(self, element):

        current_index = self.df.loc[self.df.date ==  element.date].index[0]
        current_close = element.close
        current_candle = element.candle



        if current_index > 0:
            prev_index = current_index - 1 
#           prev candle colour
            prev_candle = self.df.iloc[prev_index]['candle']

#           current candle colour
            current_candle = self.df.iloc[current_index]['candle']
            
            if current_candle == 'G':
#                 Both green current close must be higher
                if prev_candle == 'G' and self.df.iloc[current_index]['close'] > self.df.iloc[prev_index][ 'close']:
                    return 1
                
#                If last candle Red then entry candle must close above last candle open
                elif prev_candle == 'R' and self.df.iloc[current_index]['close'] > self.df.iloc[prev_index][ 'open']:
                    return 1

                else:
                    return 0
                
            elif current_candle == 'R':
#                 Both red current close must be lower
                if prev_candle == 'R' and self.df.iloc[current_index]['close'] < self.df.iloc[prev_index][ 'close']:
                    return 1
                
#                If last candle green then entry candle must close below last candle open
                elif prev_candle == 'G' and self.df.iloc[current_index]['close'] < self.df.iloc[prev_index][ 'open']:
                    return 1

                else:
                    return 0
                
    #     How to check for the 1st candle?
        return 0
    
    
#   SL Method

    def set_sl(self, df):
        '''Set SL 1% above/below to entry candle'''
        
        if df['pos_trade'] == '1':
            
            candle_size = df['high'] - df['low']
            
            if df['candle'] == 'G':
                
                sl = df['low'] - (candle_size/100)
            
            elif df['candle'] == 'R':
                
                sl = df['high'] + (candle_size/100)
                
            return sl
        
        else:
            return 0
        
    
    def check_rsi(self, df):
        '''To check RSI and EMAs'''
        
#         ONLY TRADE WHEN 20 EMA IN FAVOUR

        if (df['candle'] == 'G' and df['RSI'] <= 68 and df['close'] > df['EMA_200'] and 
            df['trend'] == 'up' and df['EMA_20'] > df['EMA_200'] ):
            
                return 1
            
            
        elif (df['candle'] == 'R' and df['RSI'] >= 32 and df['close'] < df['EMA_200'] and 
              df['trend'] == 'down' and df['EMA_20'] < df['EMA_200'] ):
        
                return 1
        
 
        else:
            
            return 0
            
            
# To mark trades as Buy/ Sell

    def check_trade(self, df):
        
        if df['candle'] == 'G':
            
            return 'B'
        
        else:
            
            return 'S'
        
#  Main Calc method

    def mark_emas(self):
        '''Marking candles with colour'''
        
        self.df['candle'] = self.df.apply(self.candle, axis=1)
        
        '''Marking candles on emas'''
        
        self.df['pos_trade'] = self.df.apply(lambda x: '1' if  (
                               
                                x['high'] < x['upper_keltner'] and x['low'] > x['lower_keltner']
            
                                )
                                else '0', axis=1)
        

#     calculate climax with
#     RSI OB/OS and candle body outside keltner's channel extreme
        
        count_candles = 0
        
#       
        
    
#     For counting 20 candles after climax

        buy_flag = True
        sell_flag = True
        green_count = 0
        red_count = 0
        
        self.df['tradable'] = 0
        
        for index, row in self.df.iterrows():
            
            one_percent = row['open']/100 
            
#             When green candle goes outside channel
            if ( ((row['candle'] == 'G' and row['open'] > row['upper_keltner']) or
                
                (row['candle'] == 'R' and row['close'] > row['upper_keltner'])) and
                
                sell_flag == True and 
                
                ( abs(row['close'] - row['upper_keltner'])/one_percent ) >= 0.07 ):
        
                
                buy_flag = False
                green_count = 0
                self.df.at[index,'tradable'] = 'First G Break'
                
#             When price remain outside after green breakout
            elif( row['close'] > row['upper_keltner'] and sell_flag == True and 
                 buy_flag == False and green_count == 0):
                
                self.df.at[index,'tradable'] = 'Outside after G Break'
                

#            Any candle in 20 count closes outside channel, restart the count

            elif row['close'] > row['upper_keltner'] and buy_flag == True and green_count > 0:
                
                green_count = 0
                buy_flag = False
                self.df.at[index,'tradable'] = 'Outside after G Break'
            
#             When price returns inside channel after buy breakout
            elif row['close'] < row['upper_keltner'] and buy_flag == False:
                
                buy_flag = True
                green_count = 1
                self.df.at[index,'tradable'] = 'G count 1'
                
#                 Counting 20 candles when green breakout moves inside channel
            elif green_count > 0 and green_count < 19 and row['close'] < row['upper_keltner']:
                
                green_count += 1
                self.df.at[index,'tradable'] = 'Counting after G Break'
               
#             When 20 candles after climax are over
            elif green_count == 19 and row['close'] < row['upper_keltner'] and red_count == 0:
                
                green_count = 0
                self.df.at[index,'tradable'] = 0
                
                
#                 =============================================
                
#             When red candle goes outside channel
            if ( ((row['candle'] == 'R' and row['open'] < row['lower_keltner'] ) or
                 
                (row['candle'] == 'G' and row['close'] < row['lower_keltner']) )and 
                
                buy_flag == True  and
                
                ( abs(row['lower_keltner'] - row['close'])/one_percent ) >= 0.07 ):
                
                sell_flag = False
                red_count = 0
                self.df.at[index,'tradable'] = 'Red Breakout'
                
#             When price remain outside channel after red breakout

            elif row['close'] < row['lower_keltner'] and buy_flag == True and sell_flag == False:
                
                self.df.at[index,'tradable'] = 'Outside after Red Breakout'
                
                
#            Any candle in 20 count closes outside channel, restart the count

            elif row['close'] < row['lower_keltner'] and sell_flag == True and red_count > 0:
                
                red_count = 0
                sell_flag = False
                self.df.at[index,'tradable'] = 'Outside after R Break'
                

#             When price returns inside channel after sell breakout
            elif row['close'] > row['lower_keltner'] and sell_flag == False:
                
                sell_flag = True
                red_count = 1
                self.df.at[index,'tradable'] = 'Sell count 1'
                
#                 Counting 20 candles when red breakout moves inside channel
            elif red_count > 0 and red_count < 19 and row['close'] > row['lower_keltner']:
                
                red_count += 1
                self.df.at[index,'tradable'] = 'Counting after sell breakout'
               
#             When 20 candles after climax are over
            elif red_count == 19 and row['close'] > row['lower_keltner'] and green_count == 0:
                
                red_count = 0
                self.df.at[index,'tradable'] = 0
        
        
        
        

                
#             Calculating SL
        
        self.df['sl'] = self.df.apply(self.set_sl, axis=1)
        
#         Check trend
        self.squeeze = self.df.loc[(self.df['squeeze_on'] == False) & (self.df['pos_trade'] == '1')  & (self.df['trend'] != 'no trend')]
#         self.squeeze = self.df.loc[(self.df['squeeze_on'] == False) & (self.df['pos_trade'] == '1')]
        
        #          Check candle size
        self.squeeze['candle_size'] = self.squeeze.apply(self.check_body, axis=1)

        self.squeeze = self.squeeze.loc[self.squeeze['candle_size'] == 1]
        
        #  Check with prev candle
        self.squeeze['prev_candle'] = self.squeeze.apply(self.check_prev_candle, axis=1)
        
        self.squeeze = self.squeeze.loc[self.squeeze['prev_candle'] == 1]
        
        
#         Check RSI & EMAs
        self.squeeze['check_rsi'] = self.squeeze.apply(self.check_rsi, axis=1)
    
        self.squeeze = self.squeeze.loc[self.squeeze['check_rsi'] == 1]
        
#         mark trades as Buy or Sell
        self.squeeze['trade'] = self.squeeze.apply(self.check_trade, axis=1)
    
      
#         Check Climax
        self.squeeze = self.squeeze.loc[self.squeeze['tradable'] == 0]
        
        


In [16]:
currency = BackTest(['USDJPYm', 6.82682, 24000])

# Record the end time
# start_time = datetime.datetime.now()



# currency.indicators()
# currency.mark_emas()


# # Record the end time
# end_time = datetime.datetime.now()

# # Calculate the elapsed time
# elapsed_time = end_time - start_time

# print(elapsed_time.seconds)

# currency.display()

Working on:  USDJPYm
